In [134]:
import numpy as np
from progressbar import *
from rich.progress import track
import matplotlib.pyplot as plt
import random
from PIL import Image
from dataset import Classification_Dataset
from sklearn.model_selection import train_test_split
from hydra import compose, initialize
from omegaconf import OmegaConf
from transforms import *
from torch.utils.data import DataLoader
import pandas as pd
from collections import defaultdict
import time
import torch
import gc
import torch.optim as optim
from torch.optim import lr_scheduler

import hydra
import numpy as np
from omegaconf import DictConfig, OmegaConf
from torch import nn

from tqdm import tqdm
import wandb
from torch import nn
from collections import OrderedDict
from torchvision import models


In [135]:
labels = {'cane': 'dog',
'cavallo': 'horse',
'elefante': 'elephant',
'farfalla': 'butterfly',
'gallina': 'chicken',
'gatto': 'cat',
'mucca': 'cow',
'pecora': 'sheep',
'ragno': 'spider',
'scoiattolo': 'squirrel',
}

config = {  'epochs': 100,
            'lr': 0.0008,
            'path': '/home/jarybski/Desktop/Animal-10/archive/raw-img',
            'img_size': 256,
            'batch_size': 8,
            'shuffle': True,
            'pin_memory': False,
            'random_state':192,
}



In [136]:
from torchvision import transforms
def transform_train(cfg):
    data_transform = transforms.Compose([transforms.Resize(224),
                                         transforms.RandomHorizontalFlip(p=0.5),
                                         transforms.RandomVerticalFlip(p=0.5),
                                         transforms.Normalize(mean=255,std=1),
                                         transforms.ToTensor(),
                                        ])
    return data_transform
    
def transform_val(cfg):
    data_transform = transforms.Compose([transforms.Resize(cfg.dataset.img_size),
                                         transforms.Resize(224),
                                         transforms.ToTensor()
                                        ])
    return data_transform




In [137]:

def get_labels():
    labels = {'cane': 'dog',
    'cavallo': 'horse',
    'elefante': 'elephant',
    'farfalla': 'butterfly',
    'gallina': 'chicken',
    'gatto': 'cat',
    'mucca': 'cow',
    'pecora': 'sheep',
    'ragno': 'spider',
    'scoiattolo': 'squirrel',
    }
    return labels

def create_df():
    cfg = OmegaConf.load("cfg/config.yaml")
    cfg_ds = cfg.dataset
    data = {'path': [],'label': []}
    labels = get_labels()

    for  dir in os.listdir(cfg_ds.path):
        filename = os.listdir(f'{cfg_ds.path}/{dir}')    
        for file in filename:
            data['path'].append(f'{cfg_ds.path}/{dir}/{file}')
            data['label'].append(labels[dir])
    df_data = pd.DataFrame(data)
    return df_data, labels


def create_dataloaders():
    cfg = OmegaConf.load("cfg/config.yaml")

    cfg_ds = cfg.dataset
    df, labels = create_df()
    df_train, df_valid = train_test_split(df, test_size=0.2, random_state=cfg.dataset.random_state, stratify=df['label'])
    
    train_dataset = Classification_Dataset(dataframe=df_train,labels=labels, shuffle=True, transforms=transform_train(cfg))
    print(len(train_dataset))
    print(train_dataset.__sizeof__())
    val_dataset = Classification_Dataset(dataframe=df_valid,labels=labels, shuffle=True, transforms=transform_val(cfg))
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=cfg_ds.batch_size, 
        shuffle=cfg_ds.shuffle, 
        num_workers=cfg_ds.num_workers,
        pin_memory=cfg_ds.pin_mem
    )

    val_loader = DataLoader(
        val_dataset, 
        batch_size=cfg_ds.batch_size,                       
        shuffle=cfg_ds.shuffle, 
        num_workers=cfg_ds.num_workers,
        pin_memory=cfg_ds.pin_mem
        )

    return train_loader, val_loader, labels, train_dataset


In [138]:
def show_images(df):
    plt.figure(figsize = (20, 6))
    for idx, i in enumerate(df.label.unique()):
        plt.subplot(1, 10, idx + 1)
        df = df[df['label'] == i].reset_index(drop=True)
        image_path = df.loc[random.randint(0, len(df) - 1), 'path']
        img = Image.open(image_path)
        img = img.resize((224,224))
        plt.imshow(img)
        plt.axis('off')
        plt.title(i)

    plt.tight_layout()
    plt.show()



In [139]:
train_loader, valid_loader, labels, train_dataset = create_dataloaders()
print("Train loader", train_loader)
print("Validation loader", valid_loader)

20902
32
Train loader <torch.utils.data.dataloader.DataLoader object at 0x7f45f0f3e380>
Validation loader <torch.utils.data.dataloader.DataLoader object at 0x7f45f0f3ed10>


In [154]:
import torch
import torchvision.transforms.functional as F


# for i in range(10):
#     item = train_dataset.__getitem__(i)
#     print(type(item))
    # a = torch.FloatTensor(item,224,224)
    # a = F.to_pil_image(a)
#print(item.size())
for batch_idxc, data in enumerate(train_loader):    
    print(type(data))

['horse']['butterfly']

[0][6]
['dog']

[4]
['squirrel']
[9]
['spider']['spider']

[8][8]

['butterfly']
[0]['sheep']

[7]
['chicken']['dog']

[4]
['spider']
[2]
[8]
['cat']
['dog'][1]
['squirrel']
[4]

[9]['spider']
['cow']
[8]

[3]['squirrel']

[9]
['chicken']
[2]['cow']
[3]

['cow']['butterfly']


[0]['elephant'][3]

['squirrel']


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/jarybski/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/jarybski/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/home/jarybski/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 160, in default_collate
    return elem_type({key: default_collate([d[key] for d in batch]) for key in elem})
  File "/home/jarybski/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 160, in <dictcomp>
    return elem_type({key: default_collate([d[key] for d in batch]) for key in elem})
  File "/home/jarybski/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 149, in default_collate
    return default_collate([torch.as_tensor(b) for